In [ ]:
!pip install findspark

In [ ]:
pip install pyspark


In [1]:
import os

# JAVA_HOME path
os.environ['JAVA_HOME'] = 'C:\\Users\\rshaw\\Downloads\\jdk-11.0.22_windows-x64_bin'

In [2]:
import os

# SPARK_HOME path
os.environ['SPARK_HOME'] = 'C:\\Users\\rshaw\\Downloads\\spark-3.5.1-bin-hadoop3\\spark-3.5.1-bin-hadoop3'

import findspark
findspark.init()


In [ ]:
# Read CSV 
file_path = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
home_sales_df = spark.read.csv(file_path, header=True, inferSchema=True)

# Create table "home_sales"
home_sales_df.createOrReplaceTempView("home_sales")


In [ ]:
# What is the average price for a four-bedroom house sold for each year
query1 = """
SELECT year, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year
ORDER BY year
"""

avg_price_4_bedrooms = spark.sql(query1)
avg_price_4_bedrooms.show()



In [ ]:
# What is the average price of a home for each year the home was built, that has three bedrooms and three bathrooms?
query2 = """
SELECT year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY year_built
ORDER BY year_built
"""

avg_price_3_bed_3_bath = spark.sql(query2)
avg_price_3_bed_3_bath.show()



In [ ]:
# What is the average price of a home for each year the home was built, that has three bedrooms, three bathrooms, two floors, and is greater than or equal to 2,000 square feet?
query3 = """
SELECT year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND area >= 2000
GROUP BY year_built
ORDER BY year_built
"""

avg_price_3_bed_3_bath_2_floors = spark.sql(query3)
avg_price_3_bed_3_bath_2_floors.show()


In [ ]:
# What is the average price of a home per "view" rating having an average home price greater than or equal to $350,000?

query = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE view IS NOT NULL AND price >= 350000
GROUP BY view
ORDER BY view
"""
avg_price_per_view = spark.sql(query)
avg_price_per_view.show()


In [ ]:
# Cache home_sales table
spark.catalog.cacheTable("home_sales")

# Check 
is_cached = spark.catalog.isCached("home_sales")
print("Is home_sales cached:", is_cached)



In [ ]:
# Query the average price of a home per "view" rating using cached data
query_cached = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE view IS NOT NULL AND price >= 350000
GROUP BY view
ORDER BY view
"""



In [ ]:
# Run query w/ cached data and measure runtime
start_time_cached = time.time()
avg_price_per_view_cached = spark.sql(query_cached)
avg_price_per_view_cached.show()
end_time_cached = time.time()
runtime_cached = end_time_cached - start_time_cached
print("Runtime with cached data:", runtime_cached, "seconds")

# Partition by "date_built" 
parquet_file_path = "path_to_save_parquet_files"
home_sales_df.write.partitionBy("date_built").parquet(parquet_file_path)

# Create a table for parquet data
parquet_df = spark.read.parquet(parquet_file_path)
parquet_df.createOrReplaceTempView("home_sales_parquet")



In [ ]:
# Query to calculate average price of a home per "view" rating using Parquet data
query_parquet = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales_parquet
WHERE view IS NOT NULL AND price >= 350000
GROUP BY view
ORDER BY view
"""

# Run the query and measure runtime
start
